In [1]:
using Pkg
using JuMP
using Gurobi
#using GLPK
using XLSX
using DataFrames

In [2]:
shared_cand = DataFrame(XLSX.readtable("CandLine.xlsx", "Taul1", header=true)...) #Dataframe of shared candidate lines
int_cand =  DataFrame(XLSX.readtable("CandLineInt.xlsx", "Taul1", header=true)...) #Dataframe of internal candidate lines
shared_ex =  DataFrame(XLSX.readtable("SharedEline.xlsx", "Taul1", header=true)...) #Dataframe of shared existing lines
int_ex = DataFrame(XLSX.readtable("Tran.xlsx", "Taul1", header=true)...) #Dataframe of internal existing lines
gen =  DataFrame(XLSX.readtable("Gen.xlsx" , "Taul1", header=true)...) #Dataframe of generators
load =  DataFrame(XLSX.readtable("Load.xlsx", "Taul1", header=true)...) #Dataframe of loads
##An example of the load dataframe can be found as follows

,zoneNum,lNodeID,P_load1,P_load2,P_load3,P_load4
,Any,Any,Any,Any,Any,Any
1,1,2,-21.7,-10.85,-16.275,-5.425
2,1,3,-94.2,-47.1,-70.65,-23.55
3,1,4,-47.8,-23.9,-35.85,-11.95
4,1,5,-7.6,-3.8,-5.7,-1.9
5,1,6,-11.2,-5.6,-8.4,-2.8
6,1,9,-29.5,-14.75,-22.125,-7.375
7,1,10,-9,-4.5,-6.75,-2.25
8,1,11,-3.5,-1.75,-2.625,-0.875
9,1,12,-6.1,-3.05,-4.575,-1.525


In [3]:
cand_from(i) = (shared_cand.nodeZone1 .== i)
cand_to(i) = (shared_cand.nodeZone2 .== i)
ex_from(i) = (shared_ex.nodeZone1 .== i)
ex_to(i) = (shared_ex.nodeZone2 .== i)

ex_to (generic function with 1 method)

In [4]:
l(i) = load[load.zoneNum .== i, :] # load within zone i
g(i) = gen[gen.zoneNum .== i, :]   # generators within zone i
shared_c(i) = vcat(shared_cand[shared_cand.nodeZone1 .== i,:] , shared_cand[shared_cand.nodeZone2 .== i, :]) #shared candidate lines within zone i
int_c(i) = int_cand[int_cand.zoneNum .== i, :]   # number of internal candidate lines within zone i
shared_e(i) = vcat(shared_ex[shared_ex.nodeZone1 .== i,:] , shared_ex[shared_ex.nodeZone2 .== i, :]) #shared existing lines within zone i
int_e(i) =int_ex[int_ex.zoneNum .== i, :]       # internal existing lines within zone i
MC(i) = (g(i).C2 .* (g(i).PgMax .^ 2) .+ g(i).C1 .* g(i).PgMax .- g(i).C2 .*(g(i).PgMin .^ 2) .- g(i).C1 .* g(i).PgMin) ./ (g(i).PgMax .- g(i).PgMin) #Marginal cost of generators within zone i
bin_c(i) = (shared_cand.nodeZone1 .== i) + (shared_cand.nodeZone2 .== i) # A binary vector through which we can check if the shared candidate lines belong to zone i
bin_e(i) = (shared_ex.nodeZone1 .== i) + (shared_ex.nodeZone2 .== i) # A binary vector through which we can check if the shared existing lines belong to zone i

bin_e (generic function with 1 method)

In [5]:
m(i) = setdiff([1,2,3], [i])

m (generic function with 1 method)

In [6]:
function zonal_app(i, N, cand_lambda, ex_lambda,shared_line_decision_var, int_line_decision_var,
        shared_cand_angle_self, shared_ex_angle_self, shared_cand_angle_1, shared_ex_angle_1, 
        shared_cand_angle_2, shared_ex_angle_2)
    
    Mod4 = Model(Gurobi.Optimizer)
    
    
    @variable(Mod4,0 <= gen_var[1:N]) # generation at each node
    @variable(Mod4,shared_cand_flow[1:nrow(shared_cand)])  #Power flowing into shared candidate lines 
    @variable(Mod4,int_cand_flow[1:nrow(int_c(i))]) #Power flowing into internal candidate lines 
    @variable(Mod4,0 <= shared_cand_angle[1:nrow(shared_cand),1:2] <= 2*pi) #Phase angle decision for shared candidate lines
    @variable(Mod4,0 <= int_cand_angle[1:nrow(int_c(i)),1:2] <= 2*pi) #Phase angle decision for internal candidate lines
    @variable(Mod4,shared_ex_flow[1:nrow(shared_ex)])  #Power flowing on shared existing shared lines
    @variable(Mod4,int_ex_flow[1:nrow(int_e(i))])  #Power flowing on internal existing lines
    @variable(Mod4,0 <= shared_ex_angle[1:nrow(shared_ex),1:2] <= 2*pi) #Phase angle for existing shared lines
    @variable(Mod4,0 <= int_ex_angle[1:nrow(int_e(i)),1:2] <= 2*pi) #Phase angle for internal existing lines
    
    omega = 2;
    eta =1;
    
    @expression(Mod4, total_cost, sum((gen_var[n] .* sum((g(i).gNodeID .== n) .* MC(i))) for n in 1:N)
        .+ sum(sum(eta * bin_e(i) .* shared_ex_angle[h,1] .* (shared_ex_angle_self[h,1] .- shared_ex_angle_1[h,2]) .+
            eta * bin_e(i) .* shared_ex_angle[h,1] .* (shared_ex_angle_self[h,1] .- shared_ex_angle_2[h,2]) .+
            eta * bin_e(i) .* shared_ex_angle[h,2] .* (shared_ex_angle_self[h,2] .- shared_ex_angle_1[h,1]) .+
            eta * bin_e(i) .* shared_ex_angle[h,2] .* (shared_ex_angle_self[h,2] .- shared_ex_angle_2[h,1]) .+
            bin_e(i) .* shared_ex_angle[h,1] .* ex_lambda[h,1] .+ 
            bin_e(i) .* shared_ex_angle[h,2] .* ex_lambda[h,2] for h in 1:nrow(shared_ex))) .+
            sum(sum((omega /2) * bin_e(i) .* (shared_ex_angle[:,1] .- shared_ex_angle_self[:,1]) .^ 2)) .+
            sum(sum((omega /2) * bin_e(i) .* (shared_ex_angle[:,2] .- shared_ex_angle_self[:,2]) .^ 2))
        .+ sum(sum(eta * bin_c(i) .* shared_cand_angle[c,1] .* (shared_cand_angle_self[c,1] .- shared_cand_angle_1[c,2]) .+
            eta * bin_c(i) .* shared_cand_angle[c,1] .* (shared_cand_angle_self[c,1] .- shared_cand_angle_2[c,2]) .+
            eta * bin_c(i) .* shared_cand_angle[c,2] .* (shared_cand_angle_self[c,2] .- shared_cand_angle_1[c,1]) .+
            eta * bin_c(i) .* shared_cand_angle[c,2] .* (shared_cand_angle_self[c,2] .- shared_cand_angle_2[c,1]) .+
            bin_c(i) .* shared_cand_angle[c,1] .* cand_lambda[c,1] .+ 
            bin_c(i) .* shared_cand_angle[c,2] .* cand_lambda[c,2] for c in 1:nrow(shared_cand))) .+
            sum(sum((omega /2) * bin_c(i) .* (shared_cand_angle[:,1] .- shared_cand_angle_self[:,1]) .^ 2)) .+
            sum(sum((omega /2) * bin_c(i) .* (shared_cand_angle[:,2] .- shared_cand_angle_self[:,2]) .^ 2)))
            
            
            
            
    for n in 1:N
        # Power balance constraint for each node
        @constraint(Mod4, sum(g(i).gNodeID .== n) .* gen_var[n] .+ sum(l(i).P_load1 .* (l(i).lNodeID .== n)) .==
            sum((shared_cand.tNodeID1 .== n) .*bin_c(i) .* shared_cand_flow) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(i) .* shared_cand_flow) .+
            sum((shared_ex.tNodeID1 .== n) .* bin_e(i) .* shared_ex_flow) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(i) .* shared_ex_flow) .+
            sum((int_c(i).tNodeID1 .== n) .* int_cand_flow) .- sum((int_c(i).tNodeID2 .== n) .* int_cand_flow) .+
            sum((int_e(i).tNodeID1 .== n) .* int_ex_flow) .- sum((int_e(i).tNodeID2 .== n) .* int_ex_flow))
        #Lower limit for generation of each node
        @constraint(Mod4, sum(g(i).gNodeID .== n) .* gen_var[n] .<= sum((g(i).gNodeID .== n) .* g(i).PgMax))
        #Upper limit for generation of each node
        @constraint(Mod4, sum((g(i).gNodeID .== n) .* g(i).PgMin) .<= sum(g(i).gNodeID .== n) .* gen_var[n])
    end
    
    M=1000 #Big number
    
    #Constraints elated to the candidate shared lines:
    for c in 1:nrow(shared_cand) 
        #Associating the power flowing within the line with the phase angles of nodes:
        @constraint(Mod4,-M .* (1 .- bin_c(i) .* shared_line_decision_var[c]) .<= bin_c(i) .* shared_cand_flow[c] .- ((1 ./ shared_cand.reacT[c]) .* (bin_c(i) .* shared_cand_angle[c,1] .- bin_c(i) .* shared_cand_angle[c,2])))
        
        @constraint(Mod4, bin_c(i) .* shared_cand_flow[c] .- ((1 ./ shared_cand.reacT[c]) .* (bin_c(i) .* shared_cand_angle[c,1] .- bin_c(i) .* shared_cand_angle[c,2])) .<= M .* (1 .- bin_c(i) .* shared_line_decision_var[c]))
        #limiting the upper bound of power flow flowing within candidate shared lines
        @constraint(Mod4, bin_c(i) .* shared_cand_flow[c] .<= bin_c(i) .* shared_line_decision_var[c] .* shared_cand.ptMax[c])
        #Limiting the lower bound of power flowing within the candidate shared lines
        @constraint(Mod4, -shared_cand.ptMax[c] .* bin_c(i) .* shared_line_decision_var[c] .<= bin_c(i) .* shared_cand_flow[c])
        @constraint(Mod4, shared_cand_angle[5,2] .== 0)
    end
    
    #The same constraints related to the candidate internal lines:
    for c in 1:nrow(int_c(i))
        @constraint(Mod4,-M .* (1 .- int_line_decision_var[c]) .<= int_cand_flow[c] .- ((1 ./ int_c(i).reacT[c]) .* (int_cand_angle[c,1] .- int_cand_angle[c,2])))
        @constraint(Mod4, int_cand_flow[c] .- ((1 ./ int_c(i).reacT[c]) .* (int_cand_angle[c,1] .- int_cand_angle[c,2])) .<= M .* (1 .- int_line_decision_var[c]))
        @constraint(Mod4, int_cand_flow[c] .<= int_line_decision_var[c] .* int_c(i).ptMax[c])
        @constraint(Mod4, -int_c(i).ptMax[c] .*int_line_decision_var[c] .<= int_cand_flow[c])
    end
    
    #The same constraints related to the shared existing lines:
    for h in 1:nrow(shared_ex)
        @constraint(Mod4, bin_e(i) .* shared_ex_flow[h] .== (1 ./ shared_ex.reacT[h]) .* (bin_e(i) .* shared_ex_angle[h,1] .- bin_e(i) .* shared_ex_angle[h,2]))
        @constraint(Mod4, bin_e(i) .* shared_ex_flow[h] .<= bin_e(i) .* shared_ex. ptMax[h])
        @constraint(Mod4, bin_e(i) .* -shared_ex.ptMax[h] .<= bin_e(i) .* shared_ex_flow[h])
        @constraint(Mod4, shared_ex_angle[1,2] .== 0)
        @constraint(Mod4, shared_ex_angle[7,2] .== 0)

    end
    #The same constraints related to the internal existing lines:
    
    for h in 1:nrow(int_e(i))
        @constraint(Mod4, int_ex_flow[h] .== (1 ./ int_e(i).reacT[h]) .* (int_ex_angle[h,1] .- int_ex_angle[h,2]))
        @constraint(Mod4, int_ex_flow[h] .<= int_e(i).ptMax[h])
        @constraint(Mod4, -int_e(i).ptMax[h] .<= int_ex_flow[h])
    end
    
    #The model objective is to minimize the total cost:
    
    @objective(Mod4, Min, total_cost)
    optimize!(Mod4)
    if termination_status(Mod4) == MOI.OPTIMAL #If the solution is optimal then it returns the variables
    
        shared_cand_power = value.(shared_cand_flow)
        shared_cand_phase_angle = value.(shared_cand_angle)
        shared_ex_power = value.(shared_ex_flow)
        shared_ex_phase_angle = value.(shared_ex_angle)
        gen_power = value.(gen_var)
        int_cand_power = value.(int_cand_flow)
        int_cand_phase_angle = value.(int_cand_angle)
        obj_value = objective_value(Mod4)
        [obj_value, shared_cand_power,shared_cand_phase_angle,shared_ex_power,shared_ex_phase_angle,gen_power, int_cand_power, int_cand_phase_angle]

    end
end
    
    

zonal_app (generic function with 1 method)

In [7]:
cand_lambda_1= zeros(12000,nrow(shared_cand),2)
ex_lambda_1 = zeros(12000,nrow(shared_ex),2)
cand_lambda_2= zeros(12000,nrow(shared_cand),2)
ex_lambda_2 = zeros(12000,nrow(shared_ex),2)
cand_lambda_3= zeros(12000,nrow(shared_cand),2)
ex_lambda_3 = zeros(12000,nrow(shared_ex),2)

cand_angle_1 = zeros(12000,nrow(shared_cand),2)
c_angle_1 = zeros(12000,nrow(shared_cand),2)
cand_angle_2 = zeros(12000,nrow(shared_cand),2)
c_angle_2 = zeros(12000,nrow(shared_cand),2)
cand_angle_3 = zeros(12000,nrow(shared_cand),2)
c_angle_3 = zeros(12000,nrow(shared_cand),2)

ex_angle_1 = zeros(12000,nrow(shared_ex),2)
e_angle_1 = zeros(12000,nrow(shared_ex),2)
ex_angle_2 = zeros(12000,nrow(shared_ex),2)
e_angle_2 = zeros(12000,nrow(shared_ex),2)
ex_angle_3 = zeros(12000,nrow(shared_ex),2)
e_angle_3 = zeros(12000,nrow(shared_ex),2)

f=ones(12000)
shared_line_decision_var_m =  
[0.0
 0.0
 0.0
 1.0
 1.0
 1.0]
int_line_decision_var_1 =
[0.0
 1.0
 1.0
 0.0
 1.0
 1.0
 0.0
 0.0
 0.0
 1.0]
int_line_decision_var_2 =
[1.0
 0.0
 0.0
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0]
int_line_decision_var_3 =
[0.0
 0.0]

2-element Vector{Float64}:
 0.0
 0.0

In [8]:
for j in 2:100    # It considers 1000 iteration
    if abs(f[j]) > 0.01  
         
        R_1 = zonal_app(1, 14, cand_lambda_1[j,:,:], ex_lambda_1[j,:,:],shared_line_decision_var_m, int_line_decision_var_1,
        cand_angle_1[j-1,:,:], ex_angle_1[j-1,:,:], cand_angle_2[j-1,:,:],  ex_angle_2[j-1,:,:], 
        cand_angle_3[j-1,:,:], ex_angle_3[j-1,:,:])
        
        R_2 = zonal_app(2, 30, cand_lambda_2[j,:,:], ex_lambda_2[j,:,:],shared_line_decision_var_m, int_line_decision_var_2,
        cand_angle_2[j-1,:,:], ex_angle_2[j-1,:,:], cand_angle_1[j-1,:,:], ex_angle_1[j-1,:,:], 
        cand_angle_3[j-1,:,:], ex_angle_3[j-1,:,:])       
        
        R_3 = zonal_app(3, 5, cand_lambda_3[j,:,:], ex_lambda_3[j,:,:],shared_line_decision_var_m, int_line_decision_var_3,
        cand_angle_3[j-1,:,:], ex_angle_3[j-1,:,:], cand_angle_1[j-1,:,:],  ex_angle_1[j-1,:,:], 
        cand_angle_2[j-1,:,:], ex_angle_2[j-1,:,:])
        
        
        cand_lambda_1[j+1,:,:] .= cand_lambda_1[j,:,:] .+ 0.01 * (R_1[3] .- (R_2[3] .+ R_3[3]) ./2)
        cand_lambda_2[j+1,:,:] .= cand_lambda_2[j,:,:] .+ 1 * (R_2[3] .- (R_1[3] .+ R_3[3]) ./2)
        cand_lambda_3[j+1,:,:] .= cand_lambda_3[j,:,:] .+ 1 * (R_3[3] .- (R_1[3] .+ R_2[3]) ./2)
        
        
    
        ex_lambda_1[j+1,:,:] .= ex_lambda_1[j,:,:] .+ 4 * (R_1[5] .- (R_2[5] .+ R_3[5]) ./2)
        ex_lambda_2[j+1,:,:] .= ex_lambda_2[j,:,:] .+ 4 * (R_2[5] .- (R_1[5] .+ R_3[5]) ./2)
        ex_lambda_3[j+1,:,:] .= ex_lambda_3[j,:,:] .+ 4 * (R_3[5] .- (R_1[5] .+ R_2[5]) ./2)
        
        f[j] = sum(sum((R_1[3] .- R_2[3] .- R_3[3]) .^ 2)) + sum(sum((R_2[3] .- R_1[3] .- R_3[3]) .^ 2)) + sum(sum((R_3[3] .- R_1[3] .- R_2[3]) .^2)) +sum(sum((R_1[5] .- R_2[5] .- R_3[5]) .^ 2)) + sum(sum((R_2[5] .- R_1[5] .- R_3[5]) .^ 2)) + sum(sum((R_3[5] .- R_1[5] .- R_2[5]) .^2))
        
        c_angle_1 = R_1[3]
        c_angle_2 = R_2[3]
        c_angle_3 = R_3[3]
        
        e_angle_1 = R_1[5]
        e_angle_2 = R_2[5]
        e_angle_3 = R_3[5]
        if isdefined(Main, :IJulia)
            Main.IJulia.stdio_bytes[] = 0
        end
        
        print('*') 
        
    else
        print(c_angle_1)
        print(c_angle_2)
        print(c_angle_3)
        print(e_angle_1)
        print(e_angle_2)
        print(e_angle_3)
    end
end
            
        
        

Academic license - for non-commercial use only - expires 2022-02-02
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 500 rows, 146 columns and 674 nonzeros
Model fingerprint: 0xf87c4654
Model has 18 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e+01, 6e+01]
  QObjective range [2e+00, 2e+00]
  Bounds range     [6e+00, 6e+00]
  RHS range        [4e+00, 1e+03]
Presolve removed 488 rows and 115 columns
Presolve time: 0.00s
Presolved: 12 rows, 31 columns, 57 nonzeros
Presolved model has 7 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.300e+01
 Factor NZ  : 7.800e+01
 Factor Ops : 6.500e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.77095146e+06 -2.18829

In [9]:
c_angle_3 

6×2 Matrix{Float64}:
 0.0          0.0
 0.0          0.0
 0.0          0.0
 5.75         4.86113e-12
 6.28318      0.0
 4.94112e-12  6.28319

In [10]:
c_angle_2

6×2 Matrix{Float64}:
 0.0          0.0
 0.0          0.0
 0.0          0.0
 5.75         1.52076e-12
 6.28319      0.0
 8.13631e-13  6.28319

In [11]:
c_angle_1

6×2 Matrix{Float64}:
 0.0      0.0
 0.0      0.0
 0.0      0.0
 5.75     3.61945e-13
 6.28319  0.0
 0.0      6.28319

In [12]:
e_angle_3 

8×2 Matrix{Float64}:
 6.28319      0.0
 0.0          0.0
 0.0          0.0
 6.28319      6.28319
 3.6291       1.9495e-12
 6.28319      6.28319
 6.28319      0.0
 5.85552e-12  6.28319

In [13]:
e_angle_2

8×2 Matrix{Float64}:
 6.28319      0.0
 0.0          0.0
 0.0          0.0
 0.0          0.0
 6.28319      6.28319
 3.52278e-13  1.04553e-13
 6.28319      0.0
 3.6889e-13   2.87281e-13

In [14]:
e_angle_1

8×2 Matrix{Float64}:
 6.28319  0.0
 0.0      0.0
 0.0      0.0
 6.28319  6.28319
 4.61431  1.2543e-9
 0.0      0.0
 0.0      0.0
 0.0      0.0

In [15]:
bin_c(1)

6-element Vector{Int64}:
 1
 1
 1
 1
 0
 0